## Government security rate forecasting and ladder optimization 

In [1]:
#Importing necessary libraries
import os
import pandas as pd
import numpy as np
from datetime import timedelta
from statsmodels.tsa.api import VAR
from scipy.optimize import minimize
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import LSTM, Dense
from sklearn.preprocessing import MinMaxScaler

In [2]:
#Data configuration
TBILL_FILE = 'tbill.csv'
TBOND_FILE = 'tbonds.csv'
FORECAST_STEPS = 12       #Number of weeks ahead to forecast
gamma = 1.0               #Risk aversion parameter for ladder optimization
MODEL_TYPE = 'VAR'        #VAR or LSTM
LSTM_EPOCHS = 50
LSTM_BATCH = 16
LSTM_LOOKBACK = 24        #Past weeks for LSTM

In [3]:
#Clean numeric series
def clean_numeric(s: pd.Series) -> pd.Series:
    num = pd.to_numeric(s.astype(str).str.replace(',', ''), errors='coerce')
    return num.ffill().bfill()

#Date column
def find_date_column(df: pd.DataFrame) -> str:
    for col in df.columns:
        key = col.lower()
        if 'week' in key or 'date' in key or 'time' in key:
            return col
    return df.columns[0]

#T BILL data
def load_tbill(path: str) -> pd.DataFrame:
    df = pd.read_csv(path)
    date_col = find_date_column(df)
    df[date_col] = pd.to_datetime(df[date_col], dayfirst=False, errors='coerce')
    df.set_index(date_col, inplace=True)
    df = df[~df.index.duplicated(keep='last')]
    df = df.sort_index().resample('W').ffill()

    for col in df.columns:
        df[col] = clean_numeric(df[col]) / 100.0
    return df

#T BONDS data
def load_tbonds(path: str) -> pd.DataFrame:
    df = pd.read_csv(path)
    date_col = find_date_column(df)
    df[date_col] = pd.to_datetime(df[date_col], dayfirst=False, errors='coerce')
    df.set_index(date_col, inplace=True)
    df = df[~df.index.duplicated(keep='last')]
    df = df.sort_index().resample('W').ffill()

    for col in df.columns:
        df[col] = clean_numeric(df[col]) / 100.0
    return df

In [4]:
#Preparing LSTM data
def prepare_lstm_data(df: pd.DataFrame, lookback: int, forecast_steps: int):
    n_obs, n_feats = df.shape
    min_length = lookback + forecast_steps
    if n_obs < min_length:
        raise ValueError(f"Insufficient data: need at least {min_length} rows, got {n_obs}.")
    scaler = MinMaxScaler()
    scaled = scaler.fit_transform(df)
    X, y = [], []
    for i in range(n_obs - min_length + 1):
        X.append(scaled[i:i+lookback])
        y.append(scaled[i+lookback:i+min_length])
    return np.array(X), np.array(y), scaler

In [5]:
#Rate forecasting via VAR
def forecast_var(df: pd.DataFrame, steps: int) -> pd.DataFrame:
    n_obs, n_vars = df.shape
    dynamic_maxlags = min(4, max((n_obs - 1) // (n_vars + 1), 1))
    model = VAR(df)
    try:
        res = model.fit(maxlags=dynamic_maxlags, ic='aic')
    except Exception:
        res = model.fit(maxlags=dynamic_maxlags)
    lag = res.k_ar
    out = res.forecast(df.values[-lag:], steps)
    idx = pd.date_range(df.index[-1] + timedelta(weeks=1), periods=steps, freq='W')
    return pd.DataFrame(out, index=idx, columns=df.columns)

In [10]:
#Rate forecasting via LSTM
def forecast_lstm(df: pd.DataFrame) -> pd.DataFrame:
    X, y, scaler = prepare_lstm_data(df, LSTM_LOOKBACK, FORECAST_STEPS)
    X_train, y_train = X[:-1], y[:-1]
    n_steps, n_feats = X_train.shape[1], X_train.shape[2]
    model = Sequential([
        LSTM(64, input_shape=(n_steps, n_feats)),
        Dense(n_feats * FORECAST_STEPS)
    ])
    model.compile('adam', 'mse')
    model.fit(X_train, y_train.reshape(len(y_train), -1), epochs=LSTM_EPOCHS,
              batch_size=LSTM_BATCH, verbose=1)
    pred = model.predict(X[-1][np.newaxis, ...])
    out = pred.reshape(FORECAST_STEPS, n_feats)
    inv = scaler.inverse_transform(out)
    idx = pd.date_range(df.index[-1] + timedelta(weeks=1), periods=FORECAST_STEPS, freq='W')
    return pd.DataFrame(inv, index=idx, columns=df.columns)

In [11]:
#Ladder Optimization
def optimize_ladder(forecast_df: pd.DataFrame, gamma: float = 1.0) -> pd.Series:
    mu = forecast_df.mean().values
    Sigma = forecast_df.cov().values
    n = len(mu)
    cons = ({'type': 'eq', 'fun': lambda w: np.sum(w) - 1})
    bounds = [(0,1)] * n
    res = minimize(lambda w: -(w.dot(mu) - gamma * w.dot(Sigma).dot(w)),
                   np.ones(n)/n, bounds=bounds, constraints=cons)
    if not res.success:
        raise RuntimeError('Optimization failed: ' + res.message)
    return pd.Series(res.x, index=forecast_df.columns)

In [12]:
#Main execution with VAR or LSTM fallback
def main():
    tbill = load_tbill(TBILL_FILE)
    tbond = load_tbonds(TBOND_FILE)
    rates = pd.concat([tbill, tbond], axis=1).dropna()

    #Forecasting
    if MODEL_TYPE.upper() == 'VAR':
        try:
            forecast_df = forecast_var(rates, FORECAST_STEPS)
        except Exception as e:
            print(f"VAR model failed ({e}); falling back to LSTM...")
            forecast_df = forecast_lstm(rates)
    else:
        forecast_df = forecast_lstm(rates)

    print(f"Forecasted Rates (decimals):\n{forecast_df}\n")

    #Ladder optimization
    weights = optimize_ladder(forecast_df, gamma)
    print(f"Optimal Ladder Weights:\n{weights}")

    #Saving results
    forecast_df.to_csv(f'rates_forecast_{MODEL_TYPE}.csv')
    weights.to_csv('ladder_weights.csv')

if __name__ == '__main__':
    main()

VAR model failed (x contains one or more constant columns. Column(s) 15, 31, 47, 63 are constant. Adding a constant with trend='c' is not allowed.); falling back to LSTM...


C:\Users\Samudhitha\AppData\Roaming\Python\Python310\site-packages\keras\src\layers\rnn\rnn.py:200: UserWarning: Do not pass an `input_shape`/`input_dim` argument to a layer. When using Sequential models, prefer using an `Input(shape)` object as the first layer in the model instead.
  super().__init__(**kwargs)


Epoch 1/50
34/34 ━━━━━━━━━━━━━━━━━━━━ 2s 9ms/step - loss: 0.2071
Epoch 2/50
34/34 ━━━━━━━━━━━━━━━━━━━━ 0s 7ms/step - loss: 0.0519
Epoch 3/50
34/34 ━━━━━━━━━━━━━━━━━━━━ 0s 7ms/step - loss: 0.0289
Epoch 4/50
34/34 ━━━━━━━━━━━━━━━━━━━━ 0s 7ms/step - loss: 0.0200
Epoch 5/50
34/34 ━━━━━━━━━━━━━━━━━━━━ 0s 8ms/step - loss: 0.0170
Epoch 6/50
34/34 ━━━━━━━━━━━━━━━━━━━━ 0s 7ms/step - loss: 0.0131
Epoch 7/50
34/34 ━━━━━━━━━━━━━━━━━━━━ 0s 7ms/step - loss: 0.0124
Epoch 8/50
34/34 ━━━━━━━━━━━━━━━━━━━━ 0s 7ms/step - loss: 0.0109
Epoch 9/50
34/34 ━━━━━━━━━━━━━━━━━━━━ 0s 8ms/step - loss: 0.0107
Epoch 10/50
34/34 ━━━━━━━━━━━━━━━━━━━━ 0s 9ms/step - loss: 0.0100
Epoch 11/50
34/34 ━━━━━━━━━━━━━━━━━━━━ 0s 7ms/step - loss: 0.0089
Epoch 12/50
34/34 ━━━━━━━━━━━━━━━━━━━━ 0s 7ms/step - loss: 0.0080
Epoch 13/50
34/34 ━━━━━━━━━━━━━━━━━━━━ 0s 10ms/step - loss: 0.0080
Epoch 14/50
34/34 ━━━━━━━━━━━━━━━━━━━━ 0s 9ms/step - loss: 0.0076
Epoch 15/50
34/34 ━━━━━━━━━━━━━━━━━━━━ 0s 9ms/step - loss: 0.0078
Epoch 16/50
34/34 